In [38]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LassoCV
from IPython.display import display

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

import numpy as np
from sklearn.impute import SimpleImputer

from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import mean_squared_error,confusion_matrix


In [39]:

df = pd.read_csv(r"./data/titanic_train.csv")


In [40]:
df.head()
y = df['Survived']
X = df.drop('Survived',axis=1)
copy= X.copy()


In [41]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
numeric_cols = [cols for cols in X.columns if X[cols].dtypes != 'object']
# numeric_cols.remove('Survived')
# numeric_cols.remove('Pclass')
for cols in numeric_cols:
    X[cols]=imp.fit_transform(X[[cols]])

X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


In [42]:
X=X.drop("Cabin",axis=1)
X=X.drop("Name",axis=1)
display(X.head())


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,-1.730108,0.827377,male,-0.592481,0.432793,-0.473674,A/5 21171,-0.502445,S
1,-1.726220,-1.566107,female,0.638789,0.432793,-0.473674,PC 17599,0.786845,C
2,-1.722332,0.827377,female,-0.284663,-0.474545,-0.473674,STON/O2. 3101282,-0.488854,S
3,-1.718444,-1.566107,female,0.407926,0.432793,-0.473674,113803,0.420730,S
4,-1.714556,0.827377,male,0.407926,-0.474545,-0.473674,373450,-0.486337,S


In [43]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X['Embarked'] =imp.fit_transform(X[['Embarked']])


In [44]:
X=X.drop("PassengerId",axis=1)
X=X.drop("Ticket",axis=1)
#
cat_cols = [cols for cols in X.columns if X[cols].dtypes == 'object']

for cols in cat_cols:
    temp = pd.get_dummies(X[cols],drop_first=True)
    X = pd.concat([X,temp],axis=1)
    X = X.drop(cols,axis=1)
    print(cols)
    display(temp)
    
display(X.head())

Xt,Xv,yt,yv = train_test_split(X,y,test_size=0.30,random_state=0)



Sex


,male
0,1
1,0
2,0
3,0
4,1
...,...
886,1
887,0
888,0
889,1


Embarked


,Q,S
0,0,1
1,0,0
2,0,1
3,0,1
4,0,1
...,...,...
886,0,1
887,0,1
888,0,1
889,0,0


,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,1,0,1
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,0,0,0
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,0,0,1
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,0,0,1
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,1,0,1


In [45]:

newdf = pd.DataFrame()
def runLasso(Cvalue,Xt,yt,scoredict = {}):
    newdf=pd.DataFrame()
    las = LogisticRegression(penalty='l1',solver='liblinear',random_state=0,C=Cvalue)
    las.fit(Xt,yt)
    t=las.score(Xt,yt)
    s=las.score(Xv,yv)
    scoredict['Train'] = t
    scoredict['Val'] = s

    print(t)
    print(s)
    mse = mean_squared_error(yt,las.predict(Xt))
    scoredict['Mse'] =mse
    scoredict['Preds'] = las.predict(Xt)

    table =[(x,y) for x,y in zip(las.coef_,Xt.columns)]
    for i,cols in enumerate(Xt.columns):
        # print(cols)
        # print(las.coef_)
        dict = {cols:[las.coef_[0][i]]}
        newdf = pd.concat([newdf,pd.DataFrame(dict)],axis=1)
    
    newdf['Name'] = str(Cvalue) + "_Lasso"
    display(newdf.head())
    for t in table:
        print(f"{t[1]}: {t[0]}")
    return newdf
        

In [46]:
hyperparamsvalues = [0.001,0.03,0.05,10,100,1000]
dict_hp = {}
for i in hyperparamsvalues:
    dict_hp[i] = {}
newdf = pd.DataFrame()

#all the trials
# CvalueLogScores = {}
for i in hyperparamsvalues:
    # if i == 0.03:
    #     df = runLasso(i,Xt,yt,CvalueLogScores)
    # else:
    df = runLasso(i,Xt,yt,dict_hp[i])
    newdf=pd.concat([newdf,df],axis=0)
    display(newdf.head())

#regular one
RegularLogScores = {}
lr = LogisticRegression()
lr.fit(Xt,yt)
t=lr.score(Xt,yt)
s=lr.score(Xv,yv)

print(t)
print(s)
RegularLogScores['Train'] = t
RegularLogScores['Val'] = s
mse = mean_squared_error(yt,lr.predict(Xt))
RegularLogScores['Mse'] =mse
RegularLogScores['Preds'] = lr.predict(Xt)

table =[(x,y) for x,y in zip(lr.coef_,Xt.columns)]
tempdf=pd.DataFrame()
for i,cols in enumerate(Xt.columns):
    # print(cols)
    # print(las.coef_)
    dict = {cols:[lr.coef_[0][i]]}
    tempdf = pd.concat([tempdf,pd.DataFrame(dict)],axis=1)

tempdf['Name'] = "LogRegress"

newdf=pd.concat([newdf,tempdf],axis=0)

# df = runLasso(0.03,Xt,yt)
# newdf=pd.concat([newdf,df],axis=0)
# display(newdf.head())

# df = runLasso(0.05,Xt,yt)
# newdf=pd.concat([newdf,df],axis=0)
# display(newdf.head())

# df = runLasso(10,Xt,yt)
# newdf=pd.concat([newdf,df],axis=0)
# display(newdf.head())

# df = runLasso(100,Xt,yt)
# newdf=pd.concat([newdf,df],axis=0)
print("----------------")
display(newdf.head())


0.6115569823434992
0.6268656716417911


,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001_Lasso


Pclass: [0. 0. 0. 0. 0. 0. 0. 0.]


,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001_Lasso


0.7817014446227929
0.7985074626865671


...

,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,-0.900946,-0.53536,-0.433677,-0.083859,0.058718,-2.756449,0.027053,-0.448866,1000_Lasso


Pclass: [-0.90094588 -0.53536013 -0.43367713 -0.08385916  0.0587177  -2.75644906
  0.0270529  -0.44886559]


,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001_Lasso
0,-0.437944,-0.025662,0.000000,0.000000,0.000000,-0.954629,0.000000,0.000000,0.03_Lasso
0,-0.588456,-0.173127,-0.039617,0.000000,0.000000,-1.294230,0.000000,0.000000,0.05_Lasso
0,-0.897602,-0.532147,-0.430947,-0.082131,0.058311,-2.745822,0.021122,-0.441708,10_Lasso
0,-0.900641,-0.535067,-0.433428,-0.083702,0.058681,-2.755480,0.026509,-0.448216,100_Lasso


0.8073836276083467
0.7985074626865671
----------------


,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001_Lasso
0,-0.437944,-0.025662,0.000000,0.000000,0.000000,-0.954629,0.000000,0.000000,0.03_Lasso
0,-0.588456,-0.173127,-0.039617,0.000000,0.000000,-1.294230,0.000000,0.000000,0.05_Lasso
0,-0.897602,-0.532147,-0.430947,-0.082131,0.058311,-2.745822,0.021122,-0.441708,10_Lasso
0,-0.900641,-0.535067,-0.433428,-0.083702,0.058681,-2.755480,0.026509,-0.448216,100_Lasso


In [47]:
finaldf = newdf.copy()
finaldf.reset_index(inplace=True)
display(finaldf.head())

import plotly.express as px
import plotly.graph_objects as go

finaldf.index = finaldf["Name"]

finaldf= finaldf.drop(["Name","index"],axis=1)
cols = finaldf.columns
display(finaldf)


print(cols)

fig = go.Figure(data=[
    
    go.Bar(name=index, x=cols, y=row.map(lambda x: abs(x))) for index,row in finaldf.iterrows()]
                )
fig.show()



,index,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Name
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001_Lasso
1,0,-0.437944,-0.025662,0.000000,0.000000,0.000000,-0.954629,0.000000,0.000000,0.03_Lasso
2,0,-0.588456,-0.173127,-0.039617,0.000000,0.000000,-1.294230,0.000000,0.000000,0.05_Lasso
3,0,-0.897602,-0.532147,-0.430947,-0.082131,0.058311,-2.745822,0.021122,-0.441708,10_Lasso
4,0,-0.900641,-0.535067,-0.433428,-0.083702,0.058681,-2.755480,0.026509,-0.448216,100_Lasso


,Pclass,Age,SibSp,Parch,Fare,male,Q,S
Name,,,,,,,,
0.001_Lasso,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.03_Lasso,-0.437944,-0.025662,0.000000,0.000000,0.000000,-0.954629,0.000000,0.000000
0.05_Lasso,-0.588456,-0.173127,-0.039617,0.000000,0.000000,-1.294230,0.000000,0.000000
10_Lasso,-0.897602,-0.532147,-0.430947,-0.082131,0.058311,-2.745822,0.021122,-0.441708
100_Lasso,-0.900641,-0.535067,-0.433428,-0.083702,0.058681,-2.755480,0.026509,-0.448216
1000_Lasso,-0.900946,-0.535360,-0.433677,-0.083859,0.058718,-2.756449,0.027053,-0.448866
LogRegress,-0.868881,-0.512990,-0.407167,-0.070907,0.066148,-2.600159,0.058272,-0.419400


Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S'], dtype='object')


In [48]:

#0.03
for keys in dict_hp:
    print(keys)
    d = dict_hp[keys]
    for keys in d:
        print(keys)
        print(d[keys])
#regular log
print("regular")
for keys in RegularLogScores:
    print(RegularLogScores[keys])
    



0.001
Train
0.6115569823434992
Val
0.6268656716417911
Mse
0.3884430176565008
Preds
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 